<a href="https://colab.research.google.com/github/RodrigoGuedesDP/IA/blob/main/IA_Fundamentals/Trabajo_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import sys
import random
import math
import csv
import time
import os

pygame.init()
# Parámetros generales
WIDTH, HEIGHT = 800, 600
FPS = 30

# Directorio de imágenes
ASSETS_DIR = "assets"

# Función para cargar imágenes
def load_image(filename):
    return pygame.image.load(os.path.join(ASSETS_DIR, filename))

# Archivos de imagen
IMG_FONDO = "fondo.png"
IMG_ARMA = "arma.png"
IMG_F16 = "f16.png"
IMG_B2 = "b2.png"
IMG_MISIL = "misil.png"
IMG_EXPLOSION = "explosion.png"

# Parámetros SA
SA_ITER = 50
SA_TEMP_INIT = 1.0
SA_ALPHA = 0.95

# Parámetros AR (Q-Learning)
ACTIONS = ['left','right','accel','decel','straight']
STATE_X_STEP = 50
STATE_Y_STEP = 50
GAMMA = 0.9
ALPHA_Q = 0.1
EPSILON = 0.1
MAX_SPEED_DRON = 4

def load_image_colorkey(path, size=None, colorkey=(255,255,255)):
    # path aquí es directamente el Surface ya cargado por load_image, no el path str.
    # Ajustamos la función para recibir surfaces directamente.
    if isinstance(path, str):
        img = pygame.image.load(path).convert()
    else:
        img = path.convert()
    img.set_colorkey(colorkey)
    if size is not None:
        img = pygame.transform.scale(img, size)
    return img

def load_image_colorkey_black(path, frame_width, frame_height, rows, cols):
    sheet = pygame.image.load(path).convert()
    sheet.set_colorkey((0,0,0))
    frames = []
    for row in range(rows):
        for col in range(cols):
            x = col*frame_width
            y = row*frame_height
            frame = sheet.subsurface((x,y,frame_width,frame_height)).copy()
            frames.append(frame)
    return frames

class Explosion:
    def __init__(self, x, y, frames, frame_rate=5):
        self.frames = frames
        self.frame_rate = frame_rate
        self.frame_index = 0
        self.timer = 0
        self.image = self.frames[0]
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.done = False

    def update(self):
        self.timer += 1
        if self.timer % self.frame_rate == 0:
            self.frame_index += 1
            if self.frame_index >= len(self.frames):
                self.done = True
            else:
                self.image = self.frames[self.frame_index]

class Avion:
    def __init__(self, x, y, tipo='f16'):
        self.tipo = tipo
        if self.tipo == 'f16':
            f16_img = load_image(IMG_F16)
            self.original_image = load_image_colorkey(f16_img, (150,150))
            self.vx = -3
            self.vy = -1
            self.image = self.original_image
            self.rect = self.image.get_rect()
            self.rect.x = x
            self.rect.y = y
        else:
            b2_img = load_image(IMG_B2)
            self.original_image = load_image_colorkey(b2_img,(125,250))
            self.A = 50
            self.omega = 0.05
            self.vx = -2
            self.start_time = pygame.time.get_ticks()/1000.0
            self.offset_x = 30.5 - 62.5 # -32
            self.offset_y = 125 - 125 # 0
            self.angle = 0
            self.image = self.original_image
            self.rect = self.image.get_rect()
            self.rect.centerx = x - self.offset_x
            self.rect.centery = y - self.offset_y

    def update(self):
        if self.tipo=='f16':
            self.rect.x += self.vx
            self.rect.y += self.vy
        else:
            t = pygame.time.get_ticks()/1000.0 - self.start_time
            vy = self.A * math.sin(self.omega*t)
            punta_x = self.rect.centerx + self.offset_x
            punta_y = self.rect.centery + self.offset_y
            punta_x += self.vx
            punta_y += vy
            angle_img = math.degrees(math.atan2(-vy,self.vx))
            self.angle = angle_img
            rotated_image = pygame.transform.rotate(self.original_image, -self.angle)
            self.image = rotated_image
            self.rect = self.image.get_rect()
            r = math.radians(self.angle)
            ox = self.offset_x
            oy = self.offset_y
            oxr = ox*math.cos(r)-oy*math.sin(r)
            oyr = ox*math.sin(r)+oy*math.cos(r)
            self.rect.centerx = punta_x - oxr
            self.rect.centery = punta_y - oyr

class Arma:
    def __init__(self):
        arma_img = load_image(IMG_ARMA)
        self.image = load_image_colorkey(arma_img,(150,150))
        self.rect = self.image.get_rect()
        self.rect.centerx = WIDTH//2
        self.rect.centery = HEIGHT - 50

class Misil:
    def __init__(self, x, y, angle):
        misil_img = load_image(IMG_MISIL)
        self.original_image = load_image_colorkey(misil_img,(40,80))
        self.angle = angle
        self.speed = 5
        ox = 33-20
        oy = 74-40
        rotated_image = pygame.transform.rotate(self.original_image, -self.angle)
        self.image = rotated_image
        self.rect = self.image.get_rect()
        r = math.radians(self.angle)
        oxr = ox*math.cos(r)-oy*math.sin(r)
        oyr = ox*math.sin(r)+oy*math.cos(r)
        self.rect.centerx = x - oxr
        self.rect.centery = y - oyr
        rad = math.radians(self.angle)
        self.vx = self.speed * math.sin(rad)
        self.vy = -self.speed * math.cos(rad)

    def update(self):
        self.rect.x += self.vx
        self.rect.y += self.vy

class Dron:
    def __init__(self, x, y):
        misil_img = load_image(IMG_MISIL)
        self.original_image = load_image_colorkey(misil_img,(40,80))
        self.angle = 0
        self.speed = 0
        self.image = self.original_image
        self.rect = self.image.get_rect()
        self.rect.center = (x,y)
        self.max_speed = MAX_SPEED_DRON

    def update(self):
        rad = math.radians(self.angle)
        vx = self.speed*math.cos(rad)
        vy = -self.speed*math.sin(rad)
        self.rect.x += vx
        self.rect.y += vy
        if self.rect.x<0: self.rect.x=0
        if self.rect.x>WIDTH:self.rect.x=WIDTH
        if self.rect.y<0:self.rect.y=0
        if self.rect.y>HEIGHT:self.rect.y=HEIGHT

    def set_action(self, action):
        if action=='left':
            self.angle=(self.angle+10)%360
        elif action=='right':
            self.angle=(self.angle-10)%360
        elif action=='accel':
            if self.speed<self.max_speed:
                self.speed+=1
        elif action=='decel':
            if self.speed>0:
                self.speed-=1
        elif action=='straight':
            pass

class Game:
    def __init__(self, render=True, generation=1):
        self.render=render
        self.generation=generation

        # Si render es True, creamos ventana; si no, solo Surface
        if self.render:
            self.screen = pygame.display.set_mode((WIDTH,HEIGHT))
            pygame.display.set_caption(f"Simulación Intercepción - Gen {generation}")
        else:
            self.screen = pygame.Surface((WIDTH, HEIGHT))

        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont(None,24)

        # Cargar fondo
        fondo_img = load_image(IMG_FONDO)
        self.fondo = fondo_img.convert()
        self.fondo.set_colorkey((255,255,255))
        self.fondo=pygame.transform.scale(self.fondo,(WIDTH,HEIGHT))

        self.arma = Arma()
        self.aviones=[]
        self.misiles=[]
        self.dron=Dron(WIDTH//2,HEIGHT-100)

        self.aviones_restantes=100
        self.cantidad_total=100
        self.aviones_derribados=0
        self.estado='jugando'
        self.last_avion=None

        self.sa_iterations=SA_ITER
        self.sa_temp_init=SA_TEMP_INIT
        self.sa_alpha=SA_ALPHA

        self.q_table={}
        self.gamma=GAMMA
        self.alpha=ALPHA_Q
        self.epsilon=EPSILON

        self.old_s=None
        self.old_a=None
        self.current_fitness=6.0
        self.aviones_lanzados=0
        self.misil_disparado_para_este_f16=False

        explosion_img_path = os.path.join(ASSETS_DIR, IMG_EXPLOSION)
        self.explosion_frames = load_image_colorkey_black(explosion_img_path,192,192,4,5)
        self.explosions = []

        self.frame_count=0

    def state_to_key(self, dx,dy,dspeed,dangle):
        return(dx,dy,dspeed,dangle)

    def discretize_state(self,avion_evasivo):
        if avion_evasivo is None:
            return(0,0,int(self.dron.speed),int(self.dron.angle//30)%12)
        dx=avion_evasivo.rect.x-self.dron.rect.x
        dy=avion_evasivo.rect.y-self.dron.rect.y
        dx_cell=int(dx/STATE_X_STEP)
        dy_cell=int(dy/STATE_Y_STEP)
        dspeed=int(self.dron.speed)
        dangle=int(self.dron.angle//30)%12
        return self.state_to_key(dx_cell,dy_cell,dspeed,dangle)

    def q_get(self,s,a):
        return self.q_table.get((s,a),0.0)
    def q_set(self,s,a,val):
        self.q_table[(s,a)]=val
    def q_best_action(self,s):
        best_a=random.choice(ACTIONS)
        best_v=-9999
        for a in ACTIONS:
            v=self.q_get(s,a)
            if v>best_v:
                best_v=v
                best_a=a
        return best_a,best_v
    def choose_action(self,s):
        if random.random()<self.epsilon:
            return random.choice(ACTIONS)
        else:
            a,_=self.q_best_action(s)
            return a

    def lanzar_avion(self):
        tipo='f16' if self.aviones_lanzados<self.cantidad_total//2 else 'b2'
        self.aviones_lanzados+=1
        if tipo=='f16':
            y=random.randint(400,500)
            x=WIDTH+50
            avion=Avion(x,y,'f16')
        else:
            y=random.randint(0,200)
            x=WIDTH+50
            avion=Avion(x,y,'b2')
        self.aviones.append(avion)
        self.last_avion=avion
        self.misil_disparado_para_este_f16=False

    def update(self):
        if self.aviones_restantes>0:
            if self.last_avion is None:
                self.lanzar_avion()
                self.aviones_restantes-=1
            else:
                if self.last_avion.rect.x<138 and self.aviones_restantes>0:
                    self.lanzar_avion()
                    self.aviones_restantes-=1

        avion_evasivo=None
        avion_f16=None
        for a in self.aviones:
            if a.tipo=='b2':
                avion_evasivo=a
            if a.tipo=='f16':
                avion_f16=a

        old_s=self.discretize_state(avion_evasivo)
        action=self.choose_action(old_s)
        self.dron.set_action(action)

        old_s_for_q=self.old_s
        old_a_for_q=self.old_a

        for avion in self.aviones[:]:
            avion.update()
            if avion.rect.x<-100 or avion.rect.y>HEIGHT+100 or avion.rect.y<-100:
                self.aviones.remove(avion)

        for misil in self.misiles[:]:
            misil.update()
            hit_avion=None
            for avion in self.aviones:
                if misil.rect.colliderect(avion.rect):
                    hit_avion=avion
                    break
            if hit_avion:
                self.aviones.remove(hit_avion)
                self.misiles.remove(misil)
                self.aviones_derribados+=1
                ex=Explosion(misil.rect.centerx,misil.rect.centery,self.explosion_frames)
                self.explosions.append(ex)
            else:
                if(misil.rect.x<-50 or misil.rect.x>WIDTH+50 or
                   misil.rect.y<-50 or misil.rect.y>HEIGHT+50):
                    self.misiles.remove(misil)

        self.dron.update()

        for ex in self.explosions[:]:
            ex.update()
            if ex.done:
                self.explosions.remove(ex)

        new_s=self.discretize_state(avion_evasivo)
        reward=self.calculate_reward(avion_evasivo)
        if old_s_for_q is not None and old_a_for_q is not None:
            old_q=self.q_get(old_s_for_q,old_a_for_q)
            _,best_val=self.q_best_action(new_s)
            new_q=old_q+self.alpha*(reward+self.gamma*best_val -old_q)
            self.q_set(old_s_for_q,old_a_for_q,new_q)

        self.old_s=new_s
        self.old_a=action

        if avion_f16 and len(self.misiles)==0 and not self.misil_disparado_para_este_f16:
            self.disparar_misil()
            self.misil_disparado_para_este_f16=True

    def calculate_reward(self,avion_evasivo):
        if avion_evasivo is None:
            return 0
        dist=math.sqrt((avion_evasivo.rect.x - self.dron.rect.x)**2+(avion_evasivo.rect.y - self.dron.rect.y)**2)
        if dist<50:
            return 10
        elif dist<200:
            return 1
        else:
            return -1

    def draw(self):
        # Dibujado solo si render=True, aunque el código ya prepara la screen siempre
        self.screen.blit(self.fondo,(0,0))
        self.screen.blit(self.arma.image,self.arma.rect)
        for avion in self.aviones:
            self.screen.blit(avion.image,avion.rect)
        for misil in self.misiles:
            self.screen.blit(misil.image,misil.rect)
        self.screen.blit(self.dron.image,self.dron.rect)
        for ex in self.explosions:
            self.screen.blit(ex.image,ex.rect)

        panel_width=220
        panel_height=70
        panel_x=WIDTH - panel_width
        panel_y=HEIGHT - panel_height
        panel_surface=pygame.Surface((panel_width,panel_height),pygame.SRCALPHA)
        panel_surface.fill((0,0,0,180))
        self.screen.blit(panel_surface,(panel_x,panel_y))

        exito_porcentaje=0.0
        if self.cantidad_total>0:
            exito_porcentaje=(self.aviones_derribados/self.cantidad_total)*100

        text_fitness=f"Fitness(actual): {self.current_fitness:.1f} m"
        text_exito=f"%Éxito: {exito_porcentaje:.1f}%"
        t1=self.font.render(text_fitness,True,(255,255,255))
        t2=self.font.render(text_exito,True,(255,255,255))
        self.screen.blit(t1,(panel_x+10,panel_y+10))
        self.screen.blit(t2,(panel_x+10,panel_y+35))

        if self.render:
            pygame.display.flip()

            # Guardar frames solo si render=True
            if self.render and self.frame_count<600:
                os.makedirs("frames",exist_ok=True)
                filename=f"frames/frame_{self.generation:03d}_{self.frame_count:04d}.png"
                pygame.image.save(self.screen, filename)
                self.frame_count+=1

    def simulated_annealing_for_angle(self,avion):
        current_angle=0
        current_fitness=self.evaluate_angle_fitness(avion,current_angle)
        T=SA_TEMP_INIT
        for i in range(SA_ITER):
            new_angle=current_angle+random.uniform(-5,5)
            new_fitness=self.evaluate_angle_fitness(avion,new_angle)
            delta=new_fitness-current_fitness
            if delta<0:
                current_angle=new_angle
                current_fitness=new_fitness
            else:
                p=math.exp(-delta/T)
                if random.random()<p:
                    current_angle=new_angle
                    current_fitness=new_fitness
            T=T*SA_ALPHA
        self.current_fitness=current_fitness
        return current_angle

    def evaluate_angle_fitness(self,avion,angle):
        mis_x=self.arma.rect.x+114
        mis_y=self.arma.rect.y+115
        speed=5
        rad=math.radians(angle)
        vx=speed*math.sin(rad)
        vy=-speed*math.cos(rad)
        dmin=999999
        xm,ym=mis_x,mis_y
        for _ in range(200):
            xm+=vx
            ym+=vy
            dist=math.sqrt((xm-avion.rect.x)**2+(ym-avion.rect.y)**2)
            if dist<dmin:
                dmin=dist
            if xm<0 or xm>WIDTH or ym<0 or ym>HEIGHT:
                break
        return dmin

    def disparar_misil(self):
        avion_objetivo=None
        for a in self.aviones:
            if a.tipo=='f16':
                avion_objetivo=a
                break
        if avion_objetivo is None:
            angle=0
        else:
            angle=self.simulated_annealing_for_angle(avion_objetivo)
        mis_x=self.arma.rect.x+114
        mis_y=self.arma.rect.y+115
        misil=Misil(mis_x,mis_y,angle)
        self.misiles.append(misil)

    def run(self):
        running=True
        start_time=pygame.time.get_ticks()
        while running:
            self.clock.tick(FPS)
            for event in pygame.event.get():
                if event.type==pygame.QUIT:
                    running=False
            if self.estado=='jugando':
                self.update()
                self.draw()
                if self.aviones_restantes==0 and len(self.aviones)==0:
                    self.estado='terminado'
            elif self.estado=='terminado':
                running=False
        # End run


if __name__=="__main__":
    generations=100
    results=[]
    os.makedirs("frames",exist_ok=True)

    for gen in range(1,generations+1):
        if gen%20==0:
            # Cada 20 generaciones con render
            game=Game(render=True,generation=gen)
        else:
            game=Game(render=False,generation=gen)

        game.run()
        porc_exito=(game.aviones_derribados/game.cantidad_total)*100
        fit=game.current_fitness
        results.append([gen,porc_exito,fit])

    # Guardar CSV
    with open("resultados_generaciones.csv","w",newline='') as f:
        writer=csv.writer(f)
        writer.writerow(["Generacion","PorcExito","Fitness"])
        for row in results:
            writer.writerow(row)

    print("Entrenamiento finalizado. Datos guardados en resultados_generaciones.csv.")
    print("Para crear el video, por ejemplo para la generación 20:")
    print("ffmpeg -framerate 30 -i frames/frame_020_%04d.png -c:v libx264 -pix_fmt yuv420p video_gen20.mp4")
